Будем использовать физическими величинами с измерениями. Модуль PhysicalQuantities сломался, не обновлялся два года, поэтому придется попробовать `pint` + `uncertainties`.

In [89]:
from pint import UnitRegistry
from uncertainties import ufloat, umath
from scipy.constants import k as k_J_per_K
from scipy.constants import pi
import sympy as sp

In [90]:
ureg = UnitRegistry()

In [91]:
H_отсчета = (ufloat(200, 1) * ureg.kilometer).to_base_units()
H_отсчета

<Quantity(200000+/-1000, 'meter')>

In [92]:
P_отсчета =  (ufloat(8, 1)* 1e-5 * ureg.pascal).to_base_units()
P_отсчета

<Quantity((8.00000+/-1.00000)e-05, 'kilogram / meter / second ** 2')>

In [93]:
H = (ufloat(70, 10) * ureg.kilometer).to_base_units()
H

<Quantity(70000.0+/-10000.0, 'meter')>

In [94]:
σ = (ufloat(8, 2) * ureg.angstrom * ureg.angstrom).to_base_units()
σ

<Quantity((8.00000+/-2.00000)e-20, 'meter ** 2')>

In [95]:
T_отсчета = (ufloat(854.4, 200) * ureg.K).to_base_units()
T_отсчета

<Quantity(854.400+/-200.000, 'kelvin')>

In [96]:
T_экзосферы = (ufloat(1000, 300) * ureg.kelvin).to_base_units()
T_экзосферы

<Quantity(1000.00+/-300.00, 'kelvin')>

Todo: разобраться, почему `ureg` не может сам разобраться с конверсией единиц измерения, и почему там константы типа Units, а не Quantities

In [97]:
k = (k_J_per_K * ureg.J / ureg.K).to_base_units()
k

<Quantity(1.380649e-23, 'kilogram * meter ** 2 / second ** 2 / kelvin')>

In [98]:
p_экз = k * T_экзосферы /  σ / H
p_экз.to('pascal')

<Quantity((2.46544+/-1.02519)e-06, 'pascal')>

In [99]:
h_экз = H_отсчета + H * umath.log( (P_отсчета / p_экз).magnitude )
h_экз.to('km')

<Quantity(443.576+/-53.212, 'kilometer')>

In [100]:
h_экз = h_экз.to_base_units()
h_экз

<Quantity(443576+/-53212, 'meter')>

----

In [101]:
G = (1 * ureg.gravitational_constant).to_base_units()
G

<Quantity(6.6743e-11, 'meter ** 3 / kilogram / second ** 2')>

In [102]:
M_земли = ufloat(5.9722e24, 1.2e21) * ureg.kg
M_земли

<Quantity((5.97220+/-0.00120)e+24, 'kilogram')>

In [103]:
R_земли = (ufloat(6.371e6, 1e4) * ureg.meter).to_base_units()
R_земли

<Quantity((6.37100+/-0.01000)e+06, 'meter')>

In [104]:
R_земли+h_экз

<Quantity((6.81458+/-0.05414)e+06, 'meter')>

In [105]:
v_esc_2 = 2*G*M_земли/(R_земли+h_экз)
v_esc_2

<Quantity((1.16985+/-0.00930)e+08, 'meter ** 2 / second ** 2')>

Разобраться, почему pint не поддерживает корни из квадратичных единиц измерения

In [106]:
v_побега_м_с = umath.sqrt(v_esc_2.magnitude)
v_побега_м_с

10815.97357436471+/-42.981716415365625

Ага, поддерживает, но не через `umath.sqrt`

In [107]:
v_esc = (2*G*M_земли/(R_земли+h_экз))**(1/2)
v_esc

<Quantity(10816.0+/-43.0, 'meter / second')>

In [108]:
v_esc.nominal_value

10815.97357436471

----

In [109]:
m, k, T = sp.symbols('m, k, T', positive=True)  # Mass, Boltzmann constant, temperature
v, theta, phi = sp.symbols('v, theta, phi', real=True)  # Spherical coordinates
m, k, T, v, theta, phi

(m, k, T, v, theta, phi)

In [110]:
норм_коэф = (m / (2 * sp.pi * k * T))**(3/2)
норм_коэф


0.353553390593274*m**1.5/(pi**1.5*T**1.5*k**1.5)

In [111]:
f = норм_коэф * sp.exp(-m * v**2 / (2 * k * T)) * v**2 * sp.sin(theta)
f

0.353553390593274*m**1.5*v**2*exp(-m*v**2/(2*T*k))*sin(theta)/(pi**1.5*T**1.5*k**1.5)

Интегрируем по $\phi$ от $-\pi$ до $\pi$

In [112]:
интеграл_phi = sp.integrate(f, (phi, -sp.pi, sp.pi))
интеграл_phi

0.707106781186548*m**1.5*v**2*exp(-m*v**2/(2*T*k))*sin(theta)/(pi**0.5*T**1.5*k**1.5)

Интегрируем по $\theta$ от $0$ до $\pi$

In [113]:
интеграл_theta = sp.integrate(интеграл_phi, (theta, 0, sp.pi))
интеграл_theta

1.4142135623731*m**1.5*v**2*exp(-m*v**2/(2*T*k))/(pi**0.5*T**1.5*k**1.5)

Тут жалко, что у `sympy` нет нативной интеграции `uncertainties`, но для реальных задач [можно помучится, переводя ufloat в AccumBounds](https://stackoverflow.com/questions/68097744/substitute-values-with-uncertainties-in-sympy)

In [114]:
def uf2ab(value):
    '''
    Возможно не очень корректно закладывать СтдОткл, но пока сойдет так
    '''
    nom = value.nominal_value  
    unc = value.std_dev  
    bounds = sp.AccumBounds(nom - unc, nom + unc) 
    return bounds


Интегрируем по $v$ от $v_{esc}$ до $\infty$

In [115]:
рез = sp.integrate(интеграл_theta, (v, v_esc.nominal_value, sp.oo))
рез


894709100454.91*m**1.5*(-3.72560570987656e-13*sqrt(pi)*T**(3/2)*k**(3/2)*(3 + 350955853.084067*m/(T*k))*erf(7648.04825956779*sqrt(m)/(sqrt(T)*sqrt(k)))/m**(3/2) + 1.11768171296297e-12*sqrt(pi)*T**(3/2)*k**(3/2)/m**(3/2) + 1.70961673591544e-8*T*k*(exp(-58492642.1806779*m/(T*k)) + 7648.04825956779*sqrt(pi)*sqrt(m)*erf(7648.04825956779*sqrt(m)/(sqrt(T)*sqrt(k)))/(sqrt(T)*sqrt(k)))/m)/(pi**0.5*T**1.5*k**1.5)

In [116]:
m_H_кг = 1.67e-27

In [117]:
P_побега = рез.subs({  T:T_экзосферы.nominal_value, 
            k:k_J_per_K,
            m:m_H_кг
            }).evalf()
P_побега            

0.00270759621084221

In [118]:
# Когда-нибудь разобраться, почему не получилось так
рез.subs({  T: uf2ab(T_экзосферы), 
            k:k_J_per_K,
            m:m_H_кг
            }).evalf()

671516.519817314*((erf(AccumBounds(2.33289685574736, 3.17920251937106))*AccumBounds(4.13495201573478, 5.63498974826775) + AccumBounds(4.07795976519327e-5, 0.00432904749357391))*AccumBounds(0.098938110525595, 0.183742205261819) + erf(AccumBounds(2.33289685574736, 3.17920251937106))*AccumBounds(-1.480792984893, -0.327780265196198) + AccumBounds(0.0275797521444306, 0.0698004668504409))/AccumBounds(700.0, 1300.0)**1.5

---

In [119]:
n_H = sp.symbols('n_H', positive=True)

норм_коэф = n_H * (m / (2 * sp.pi * k * T))**(3/2)
f = норм_коэф * sp.exp(-m * v**2 / (2 * k * T)) * v**3 * sp.sin(theta) * sp.cos(theta)
интеграл_phi = sp.integrate(f, (phi, -sp.pi, sp.pi))
интеграл_theta = sp.integrate(интеграл_phi, (theta, 0, sp.pi/2))
интеграл_theta

0.353553390593274*m**1.5*n_H*v**3*exp(-m*v**2/(2*T*k))/(pi**0.5*T**1.5*k**1.5)

In [120]:
рез = sp.integrate(интеграл_theta, (v, v_esc.nominal_value, sp.oo))
рез

2.41928749681598e+15*T**0.5*k**0.5*n_H*(2.92278938372215e-16 + 1.70961673591544e-8*m/(T*k))*exp(-58492642.1806779*m/(T*k))/(pi**0.5*m**0.5)

In [121]:
n_H_м3 = 1e11

In [122]:
Ф_м2_с = рез.subs({  T:T_экзосферы.nominal_value, 
                k:k_J_per_K,
                m:m_H_кг,
                n_H: n_H_м3
            }).evalf()
Ф_м2_с            

783525351902.222

----

In [123]:
M_атм = 29 * ureg.gram / ureg.mole  
N_A = 6.02214076e23 / ureg.mole
P_0 = 1e5 * ureg.pascal 
chi_H = 5.5e-7 * ureg.dimensionless 

In [124]:
ureg.g0

<Unit('standard_gravity')>

In [125]:
m_атм = (4 * pi * (R_земли**2 * P_0 / (1.0 * ureg.g0)) ).to_base_units()
m_атм

<Quantity((5.20121+/-0.01633)e+18, 'kilogram')>

In [127]:
N = (N_A * m_атм / M_атм).to_base_units()
N

<Quantity((1.08008+/-0.00339)e+44, 'dimensionless')>